In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/NIST_Utils-master/pymetamap")
sys.path
#/home/tay/Documents/DSA-NIST

['',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python27.zip',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/plat-linux2',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-tk',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-old',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-dynload',
 '/home/tay/.local/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/IPython/extensions',
 '/home/tay/.ipython',
 '/home/tay/Documents/DSA-NIST/NIST_Utils-master/pymetamap',
 '/home/tay/Documents/DSA-NIST/NIST_Utils-master/pymetamap']

In [3]:
import py_trees
import numpy as np
from py_trees.blackboard import Blackboard
import pandas as pd
from scipy import spatial
from sklearn.cluster import KMeans
from pandas import DataFrame
from annotator import annotator
import re
import pymetamap
import behaviours as be

In [4]:
# read RAA 2017
##docu = '/Users/sileshu/Desktop/EMSData/RAA Data 2017.xlsx'
docu = './RAA Data 2017.xlsx'
df = pd.read_excel(docu)
narratives = df['Narrative']
interventions = df['Interventions']

In [5]:
# read labelled cases
docu = './AutoLabelCases (Haydon Additions).xlsx'
df = pd.read_excel(docu)
su = 0
protocol = {}
narratives = df['Narratives']
gt = []
pts = set()
for item in df['Corrected Protocol']:
    if not pd.isnull(item):
        gt.append(item)
        pts.add(item)
        
for item in df.iterrows():
    if item[1]['Corrected Protocol'] in protocol:
        protocol[item[1]['Corrected Protocol']] += 1
    else:
        protocol[item[1]['Corrected Protocol']] = 1


In [6]:
# build concept list
from search_term import search_term

In [7]:
pro_df = pd.read_excel('./ODEMSA_Protocols_weighted.xlsx')

In [8]:
ss = set()
inter = set()
for item in pro_df['Signs&Symptoms']:
    for i in item.split(';'):
        i = (i.strip().lower()[:-1],i.strip().lower()[-1])
        ss.add(i)
for item in pro_df['Interventions']:
    for i in item.split(';'):
        i = (i.strip().lower()[:-1],i.strip().lower()[-1])
        inter.add(i)
for item in pro_df['Possible signs&symptoms additions']:
    if pd.isnull(item):
        continue
    for i in item.split(';'):
        i = (i.strip().lower()[:-1],i.strip().lower()[-1])
        ss.add(i)

In [9]:
dft = pd.DataFrame()
dft['Required Concepts'] = list(ss)
dft.to_csv('RC.csv',index = False)

In [10]:
concept_mapping = {}
for i in ss:
    cuis = search_term(i[0], page_limit = 1, check = True)
    if len(cuis) == 0:
        print i
    else:
        concept_mapping[i] = cuis

In [11]:
for i in inter:
    cuis = search_term(i, page_limit = 1, check = True)
    if len(cuis) == 0:
        print i
    else:
        concept_mapping[i] = cuis

(u'cardiac monitor', u'3')
(u'bag valve mask ventilation', u'3')
(u'cardiac monitor', u'2')
(u'suction the oropharynx', u'2')
(u'nasopharyngeal airway', u'2')
(u'bronchodilator', u'3')
(u'encourage patient to relax', u'3')
(u'endotracheal tube', u'3')
(u'recovery position', u'2')
(u'capnography', u'3')
(u'physical restraint', u'2')
(u'geodon', u'2')
(u'metered dose inhaler', u'2')
(u'suction the nasopharynx', u'1')
(u'supine position', u'2')


In [12]:
dfs = []
for i in concept_mapping:
    df = DataFrame()
    cuis = [j[0] for j in concept_mapping[i]]
    names = [j[1] for j in concept_mapping[i]]
    concept = [i[0]] + [np.nan] * (len(cuis) - 1)
    df['Required Concept'] = concept
    df['CUI'] = cuis
    df['Preferred Name'] = names
    dfs.append(df)
concept_list = pd.concat(dfs)

In [13]:
inter

{(u'albuterol', u'3'),
 (u'aspirin', u'3'),
 (u'bag valve mask ventilation', u'3'),
 (u'bronchodilator', u'3'),
 (u'capnography', u'3'),
 (u'cardiac monitor', u'2'),
 (u'cardiac monitor', u'3'),
 (u'cpap', u'3'),
 (u'dexamethasone', u'3'),
 (u'dextrose', u'3'),
 (u'diazepam', u'1'),
 (u'diazepam', u'2'),
 (u'diazepam', u'3'),
 (u'encourage patient to relax', u'3'),
 (u'endotracheal tube', u'3'),
 (u'epinephrine', u'2'),
 (u'fentanyl', u'2'),
 (u'geodon', u'2'),
 (u'glucagon', u'2'),
 (u'ipratropium', u'3'),
 (u'magnesium sulfate', u'2'),
 (u'metered dose inhaler', u'2'),
 (u'midazolam', u'1'),
 (u'midazolam', u'2'),
 (u'midazolam', u'3'),
 (u'morphine', u'2'),
 (u'narcan', u'3'),
 (u'nasopharyngeal airway', u'2'),
 (u'nitroglycerin', u'2'),
 (u'normal saline', u'2'),
 (u'ondansetron', u'3'),
 (u'oral glucose', u'3'),
 (u'oxygen', u'1'),
 (u'oxygen', u'2'),
 (u'oxygen', u'3'),
 (u'physical restraint', u'2'),
 (u'recovery position', u'2'),
 (u'suction the nasopharynx', u'1'),
 (u'suction

In [14]:
concept_list

,Required Concept,CUI,Preferred Name
0,midazolam,C1351678,Midazolam 2 MG/ML Oral Solution
1,NaN,C1126025,Midazolam 2 MG/ML
2,NaN,C2920420,2 ML Midazolam 1 MG/ML Prefilled Syringe
3,NaN,C3500195,2 ML Midazolam 1 MG/ML Cartridge
4,NaN,C4048415,2 ML Midazolam 1 MG/ML Injection
5,NaN,C0358428,Midazolam 2 MG/ML Injectable Solution
6,NaN,C3857257,2 ML Midazolam 5 MG/ML Prefilled Syringe
7,NaN,C0978647,Midazolam 1 MG/ML Injectable Solution
8,NaN,C3502910,Midazolam 2 MG/ML Prefilled Syringe
9,NaN,C3853826,1 ML Midazolam 5 MG/ML Prefilled Syringe


In [15]:
narratives

0      Dispatched for seizures    Aos to find 59 yof ...
1      A: Dispatched with RFD to club for employee wi...
2      A: 596 was dispatched to a P3 fall call. Upon ...
3      D: "diabetic problem"    A: Arrived to find a ...
4      D: Dispatched to the listed address for a psyc...
5      D- Unit 594 dispatched priority one for a brea...
6      D- Dispatched complaint: Breathing Difficulty ...
7      D:  Dispatched to the above address, a persona...
8      D- Dispatched complaint: Sick person       Oth...
9      D: Dispatched Priority 1 for hemorrhage/lacera...
10     Dispatch: Dispatched priority 1 for a breathin...
11     A: Dispatched for a female with vaginal bleedi...
12     Dispatch: Dispatched Priority 1 for the chest ...
13     [Dispatch]  M586 dispatched for the up/md    [...
14     D:  EMS dispatched for a sick person/interfaci...
15     D: 491 dispatched to sick person    A: Arrived...
16     Dispatch for a P.1 male unconscious but breath...
17     D- RAA responded immedia

In [16]:
concept_list.to_csv('concept_list(s&s)_revised.csv',index = False)

In [17]:
# cluster protocol
pv = {}
for line in pro_df.iterrows():
    sv = []
    iv = []
    line_ss = [(i.strip().lower()[:-1],i.strip().lower()[-1]) for i in line[1]['Signs&Symptoms'].split(';')]
    line_inter = [(i.strip().lower()[:-1],i.strip().lower()[-1]) for i in line[1]['Interventions'].split(';')]
    if not pd.isnull(line[1]['Possible signs&symptoms additions']):
        line_ssr = [(i.strip().lower()[:-1],i.strip().lower()[-1]) for i in line[1]['Possible signs&symptoms additions'].split(';')]
    line_ss += line_ssr
    print line_ss
    name = line[1]['Protocol']
    for i in ss:
        res = 0.
        for j in line_ss:
            if i[0] == j[0]:
                res = 2.**int(j[1])
                break;
        sv.append(res)
    #for j in inter:
     #   iv.append(int(j in line_inter))
    pv[name] = sv
            

[(u'chest pain', u'3'), (u'pain region', u'2'), (u'pain radiation', u'3'), (u'pain severity', u'3'), (u'pale', u'3'), (u'diaphoresis', u'3'), (u'shortness of breath', u'2'), (u'nausea', u'2'), (u'vomiting', u'2'), (u'dizziness', u'1'), (u'chest pressure', u'3'), (u'pain chest wall', u'2'), (u'pain jaw', u'2'), (u'chest tightness', u'2')]
[(u'abdominal pain', u'3'), (u'distension', u'3'), (u'rigidity', u'3'), (u'unequal femoral pulses', u'1'), (u'absent femoral pulses', u'1'), (u'diaphoresis', u'2'), (u'tenderness', u'2'), (u'nausea', u'3'), (u'vomiting', u'3'), (u'diarrhea', u'3'), (u'dysuria', u'3'), (u'constipation', u'3'), (u'vaginal bleeding', u'3'), (u'vaginal discharge', u'2'), (u'pregnancy', u'3'), (u'fever', u'3'), (u'headache', u'2'), (u'weakness', u'2'), (u'malaise', u'2'), (u'myalgias', u'2'), (u'cough', u'1'), (u'rash', u'1'), (u'rebound tenderness', u'3')]
[(u'anxiety', u'2'), (u'agitation', u'3'), (u'confusion', u'2'), (u'hallucinations', u'3'), (u'delusional thoughts', u

In [18]:
l = []
names = []
for i in pv:
    names.append(i)
    l.append(pv[i])
X = np.array(l)
kmeans = KMeans(n_clusters = 4, random_state=0).fit(X)

In [19]:
kmeans.labels_

array([2, 0, 0, 1, 3, 0, 3, 3], dtype=int32)

In [20]:
names

[u'Medical - Abdominal Pain',
 u'Medical - Altered Mental Status',
 u'Medical - Seizure',
 u'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
 u'General - Behavioral/Patient Restraint',
 u'Medical - Overdose/Poisoning - Opioid',
 u'Medical - Diabetic - Hypoglycemia',
 u'Medical - Chest Pain - Cardiac Suspected']

In [21]:
group = dict()
x = kmeans.labels_
for i,name in enumerate(names):
    if x[i] in group:
        group[x[i]].append(name)
    else:
        group[x[i]] = [name]

In [22]:
#Somehow make sure that the two metamap servers are on and running
#TP#!/bin/bash /public_mm/bin/skrmedpostctl start &
#TP#!/bin/bash /public_mm/bin/wsdserverctl start &

In [23]:
# extract concept and calculate similarity
from ranking_func import rank
def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
texts = ['Upper lobes bilaterally and quiet lower lobes bilaterally initial Vital Signs BP 162 over 94 pulse 120 for respirations 36 skin is warm and moistpulse oximetry 92% on 15 LPM by non-rebreather ECG']
results = []
scores = []
ConceptRes = []
for item in narratives:
    blackboard = Blackboard()
    blackboard.text = [item]
    root = py_trees.composites.Sequence("Root_1")
    IG = be.InformationGathering()
    TC = be.TextCollection()
    V = be.Vectorize()
    root.add_children([TC,IG,V])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
    print('Item: ', i)
    behaviour_tree.tick_tock(
            sleep_ms=50,
            number_of_iterations=1,
            pre_tick_handler=None,
       post_tick_handler=None
        )
    scores.append(blackboard.ranking)
    ConceptRes.append(blackboard.ConcLog)
    results.append(rank(blackboard.ranking))

('Item: ', 7)


TypeError: list indices must be integers, not str

In [ ]:
Inters = []
for item in interventions:
    inter = item.strip('{}').split('}{')
    inter = [i.split(':')[-1].strip().lower() for i in inter]
    Inters.append(inter)
interset = set()
for i in Inters:
    for j in i:
        interset.add(j)

In [ ]:
protocol2inter = {
    'Medical - Chest Pain - Cardiac Suspected':['oxygen','cardiac monitor','12-lead ecg',\
                               'aspirin','midazolam (versed)','diazepam (valium)',\
                                'nitroglycerine','fentanyl','hospital contact','iv','normal saline'],
    'Medical - Abdominal Pain':['oxygen','cardiac monitor','12-lead ecg',\
                                'iv','normal saline','ondansetron (zofran)','hospital contact'],
    'General - Behavioral/Patient Restraint':['oxygen','midazolam (versed)','diazepam (valium)',\
                                              'hospital contact'],
    'General - Pain Control':['oxygen','cardiac monitor','12-lead ecg',\
                              'acetaminophen (tylenol)','iv','normal saline','fentanyl'],
    'Medical - Seizure':['nasopharyngeal airway insertio','oxygen',\
                        'iv','normal saline','midazolam (versed)','diazepam (valium)',\
                        'cardiac monitor','12-lead ecg','hospital contact'],
    'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway':['oxygen','capnography (1) first reading',\
                                                                        'capnography (2) second reading','capnography (3) final reading',\
                                                                       'assist ventilation (bvm)','endotracheal','duoneb','albuterol',
                                                                       'ipratropium (atrovent)','dexamethasone (decadron)','cpap',\
                                                                       'magnesium sulfate'],
    'Medical - Altered Mental Status':['oxygen','cardiac monitor','12-lead ecg','iv','normal saline',\
                                       'hospital contact'],
    'Medical - Diabetic - Hypoglycemia':['oxygen','cardiac monitor','12-lead ecg','iv','normal saline',\
                                       'hospital contact','dextrose 10%','dextrose 25%','dextrose 5% in 0.45% ns',\
                                        'dextrose 50%','oral glucose'],
    'Medical - Overdose/Poisoning - Opioid':['oxygen','cardiac monitor','12-lead ecg','iv','normal saline',\
                                       'hospital contact','naloxone (narcan)']
}

In [ ]:
predict_inters = []
overlap_rate = []
for idx,item in enumerate(results):
    res = []
    for j in item[:3]:
        res += protocol2inter[j]
    res = set(res)
    predict_inters.append(res)
    ct = 0
    for i in Inters[idx]:
        if i in res:
            ct += 1
    overlap_rate.append(float(ct)/float(len(Inters[idx])))

    

In [ ]:
dfo = pd.DataFrame()
dfo['Narrative'] = narratives
dfo['Intervention'] = Inters
dfo['Score'] = scores
dfo['Ranking'] = [i[:3] for i in results]
dfo['Predict Intervention'] = predict_inters
dfo['Overlap Rate'] = overlap_rate
dfo.to_excel('OverlapEval_RAA2017.xlsx',index = False)

In [ ]:
sum(overlap_rate)/len(overlap_rate)

In [ ]:
dfr = DataFrame()
dfr['Narratives'] = narratives
dfr['Concepts'] = ['\n'.join(['(' + ','.join(i.strip('()').split(',')[:3]) + ')' for i in item]) for item in ConceptRes]
dfr.to_excel('ConceptExtracEval_ODEMSA.xlsx',index = False)


In [ ]:
for i in range(len(narratives)):
    if not isinstance(narratives[i], basestring):
        print i

In [ ]:
gt = df['Corrected Protocol']

In [ ]:
# rank and calculate ndcg
pool = set(['Medical - Abdominal Pain',
            'General - Pain Control',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])
def rank(scores):
    srt = sorted(scores,key=lambda x: x[1], reverse=True)
    return [i[0] for i in srt]
bina = []
ranks = []
ndcgs = []

def nDCG(rank,gt,group,num):
    from numpy import log2
    for i in group:
        if gt in group[i]:
            break
    ct = len(group[i]) - 1
    idcg = 2.
    idx = 1
    num -= 1
    while (ct&num):
        idcg += 1/log2(idx+2)
        idx += 1
        ct -= 1
        num -= 1
    
    dcg = 0.
    for idx,item in enumerate(rank):
        if item == gt:
            dcg += 2/log2(idx+2)
        elif item in group[i]:
            dcg += 1/log2(idx+2)
        else:
            continue
    return dcg/idcg

for i,s in enumerate(scores):
    res = rank(s)
    
    if gt[i] in pool:
        bina.append(gt[i] in res[:3])
        ranks.append(res)
        ndcgs.append(nDCG(res[:3],gt[i],group,3))


In [ ]:
float(sum(bina))/len(bina)

In [ ]:
float(sum(ndcgs)/len(ndcgs))

In [ ]:
t = [i[1]['Narratives'] for i in df.iterrows() if i[1]['Corrected Protocol'] in pool]

In [ ]:
len(t)

In [ ]:
len(ranks)

In [ ]:
# store results
dfr = DataFrame()
dfr['Narratives'] = t
dfr['Scores'] = [s for i,s in enumerate(scores) if gt[i] in pool]
dfr['Rank'] = ranks
dfr['GroundTruth'] = [g for g in gt if g in pool]
dfr['Correctness'] = bina

In [ ]:
dfr.to_excel('VecMethEval_ODEMSA_weighted&revised.xlsx',index = False)